In [1]:
import lightgbm as lgb
print(lgb.__version__)

3.3.4


In [1]:
# 本地
input_dir,project_dir = r"G:\\Browser\\GAIDC_preliminary_A\\", r"G:\\Browser\\GAIDC_preliminary_A\\"
# 和鲸
# input_dir,project_dir = r"/home/mw/input/rcxx20231783/", r"/home/mw/project/"
# # AIstudio
# input_dir,project_dir = r"/home/aistudio/data/data187170/", r"/home/aistudio/work/"

test_dir = input_dir + 'preliminary_A.csv'
train_dir = input_dir + 'train.csv'
prediction_dir = project_dir + 'prediction.csv'
pickle_dir = project_dir + 'df.pickle'

# 全局超参
n_jobs = 8 # 核心数
random_state = 123 # 随机数种子
n_estimators = 10000 # 迭代次数
early_stopping_rounds = int(n_estimators/10)

frac = 1 # 读取数据的比例（用于debug）

In [2]:
drop_list = ['date_id']
y_cols = 'y'

In [3]:
import sklearn
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from sklearn. model_selection import train_test_split
from tqdm import tqdm
import datetime
import time
import re
import pickle
import math



from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
from lightgbm import LGBMClassifier
import gc
from sklearn.metrics import roc_auc_score
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
import warnings
warnings.filterwarnings("ignore")

In [4]:
# df = pd.read_csv("/home/mw/input/pre8881/train.csv")#训练集
# df = pd.DataFrame(df.values[0:10000],columns=df.columns) #测试集
df_test = pd.read_csv(test_dir)
df_train = pd.read_csv(train_dir)
# df_train.dropna(thresh = len(df_train.columns) - df_train.isnull().sum(axis=1).max() + 1,inplace=True)#dropna空最多的TopN
df = df_train.append(df_test, sort=False) # .sample(frac=0.1, random_state=random_state)# 合并测试集与训练集
df = df.reset_index(drop=True) # 必须重置index否则index会重复
# del df_train,df_test
gc.collect()

53

## 数据预处理


In [5]:
# .drop(columns=['id'])
le = LabelEncoder()
for f in tqdm(list(df.select_dtypes(include='object'))):
    '''
    LabelEncoding (for object)
    '''
    try:
        
        df[f] = df[f].astype('float')# 将数字转成数值型数据
    except:
        df[f] = le.fit_transform(df[f].astype('str')).astype('int')# 字符串label化

100%|██████████| 4/4 [00:00<00:00, 48.23it/s]


In [6]:
# df.index.value_counts()
# .info()
df.loc[0]

id              48.0000
transformers     0.0000
date_id          3.0000
month            4.0000
is_weekend       0.0000
time             0.0000
L1              16.4772
L2              11.6558
L3               4.4320
L4               6.4635
L5               4.3713
L6               1.1885
y                9.7434
Name: 0, dtype: float64

## 构造衍生业务特征

In [7]:
# 数量编码
groupby_list = ['transformers', 'month', 'is_weekend', 'time', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6']
for name in tqdm(groupby_list):
    # 加上同类数属性列
    df_temp = df.groupby(name).size().reset_index()#重置索引
    df_temp.columns = [name] + ["num_in_"+name]
    df = df.merge(df_temp, how='left')  # 左合并   

100%|██████████| 10/10 [00:00<00:00, 53.01it/s]


In [8]:
L_list = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6']
L_norm_list = [i+"_norm" for i in L_list]
max_min_scaler = lambda x : (x-x.min())/(x.max()-x.min())
for i in L_list: # 将L1-6进行min-max归一化
    df[f'{i}_norm'] = df[[i]].apply(max_min_scaler).values
df["L_norm_sum"] = df[L_norm_list].sum(axis=1) # 归一化总和
df["L_sum"] = df[L_list].sum(axis=1) # 总和
df['L_norm_std']  = df[L_norm_list].std(axis=1) # 方差
df['L_std']  = df[L_list].std(axis=1) # 方差

In [9]:
# 猜测复杂L1-L6是否大于0可能会有影响
ge0 = lambda x: 1 if x>=0 else 0
L_ge0_list = [f"is_{i}_ge0" for i in L_list]
for i in L_list:
    df[f'is_{i}_ge0'] = df[[i]].applymap(ge0).values
df["num_L_ge0"] = df[L_ge0_list].sum(axis=1)
df["is_L_all_ge0"] = df["is_L1_ge0"] * df["is_L2_ge0"] * df["is_L3_ge0"] * df["is_L4_ge0"] * df["is_L5_ge0"] * df["is_L6_ge0"]

# 特征构建

In [10]:
# 统计特征用到的函数
def stat(df, df_merge, cols, agg): # 每次只计算一列cols
    # 测试集df_merge关于训练集df的cols列agg统计特征
    group = df.groupby(cols).agg(agg) #算出cols属性在df中的agg属性
    columns = []
    for on, methods in agg.items(): 
        #items为y_cols、mean;此处为字典变量————'y_cols':['mean','std']
        for method in methods: # methods也可能有多个
            if str(type(method)) == '<class \'function\'>': # 匿名函数会以地址命名，造成列名不同，所以应该更名
                method = 'mode'
            columns.append('{}_{}_{}'.format('_'.join(cols), on, method)) # 用_连接，规定分组结果group的属性名
    group.columns = columns
    group.reset_index(inplace=True)
    df_merge = df_merge.merge(group, on=cols, how='left') #以cols左连接，仅与测试集拼接
    del (group)
    gc.collect()
    return df_merge


def statis_feat(df_know, df_unknow,A_list,target_B, methods_dict):
    # 计算A关于B的统计特征：
    # df_know训练集,df_unknow验证集,A_list需计算的特征集合,target_B目标标签,methods_dict编码方式
    for f in tqdm(A_list,desc='%-30s'%target_B,leave=False):
        df_unknow = stat(df_know, df_unknow, [f], {target_B: methods_dict}) # 
    return df_unknow


def multi_groupby(df_fold_train,df_fold_val,multi_groupby_cols,target_B, methods_dict):
    # 计算测试集df_fold_val关于训练集df_fold_train的multi_groupby_cols组合列methods_dict统计特征
    multi_groupby_df1 = df_fold_train.groupby(multi_groupby_cols).agg({target_B: methods_dict})
    multi_groupby_df1.columns=["_".join(multi_groupby_cols+[target_B,"_".join(methods_dict)])]
    df_fold_val = df_fold_val.merge(multi_groupby_df1,how='left',left_on=multi_groupby_cols,right_index=True)
    return df_fold_val

# 统计特征用到的函数
def stat(df, df_merge, cols, agg): # 每次只计算一列cols
    #df练集, df_merge测试集
    # 这里只有y1参与，因此仅训练集分组
    group = df.groupby(cols).agg(agg) #算出cols属性在训练集的平均购买率(建议先了解)

    columns = []
    for on, methods in agg.items(): 
        #items为y1_is_purchase、mean;此处为字典变量————'y1_is_purchase':['mean','std']
        for method in methods: # methods也可能有多个
            if str(type(method)) == '<class \'function\'>': # 匿名函数会以地址命名，造成列名不同，所以应该更名
                method = 'mode'
            columns.append('{}_{}_{}'.format('_'.join(cols), on, method))
            #用_连接，规定分组结果group的属性名

    group.columns = columns
    group.reset_index(inplace=True)
    df_merge = df_merge.merge(group, on=cols, how='left') #以cols左连接，仅与测试集拼接
    # df['B_minus_A_B_mean']   = df['B'] - df['A_B_mean']
    # df_merge['{}_minus_{}_{}_{}_{}'.format(on,'_'.join(cols), on, method)] = df_merge[on]
    del (group)
    gc.collect()

    return df_merge


def statis_feat(df_know, df_unknow,A_list,target_B, methods_dict):
    #df_know训练集, df_unknow测试集
    for f in tqdm(A_list,desc='%-30s'%target_B,leave=False):#df_train_stas_feat
        # df.groupby(A)[B].transform(sts)称之为：A关于B的传统统计特征
        df_unknow = stat(df_know, df_unknow, [f], {target_B: methods_dict}) #  ['mean','std']

    return df_unknow

def statis_minus(df_unknow, A_list, target_B, method):
    #df_know训练集, df_unknow测试集
    for f in tqdm(A_list,desc='%-30s'%'statis_minus'):#df_train_stas_feat
        # B_subs_B_A_mean：黄金三兄弟
        # print(f'{f}_minus_{target_B}_{method}_{f}')
        df_unknow[f'{f}_minus_{target_B}_{method}_{f}'] = 0.
        # 注意如果训练集与测试集一个都没有匹配上的的话，该列可能会不存在
        try:
            df_unknow[f'{f}_minus_{target_B}_{method}_{f}'] = df_unknow[f] - df_unknow[f'{target_B}_{method}_{f}']
        except:
            df_unknow[f'{f}_minus_{target_B}_{method}_{f}'] = np.nan
    return df_unknow

def multi_groupby(df_fold_train, df_fold_val, multi_groupby_cols, target_B, methods_dict):
    multi_groupby_df1 = df_fold_train.groupby(multi_groupby_cols).agg({target_B: methods_dict})
    multi_groupby_df1.columns=["_".join(multi_groupby_cols+[target_B,"_".join(methods_dict)])]
    df_fold_val = df_fold_val.merge(multi_groupby_df1,how='left',left_on=multi_groupby_cols,right_index=True)
    return df_fold_val


In [11]:
# 组合特征目标编码
# multi_groupby_cols = [
# 	   ['month','is_weekend'],['transformers','month']]
def generate_multi_groupby_cols(target:str,rank:int)->list:
    multi_groupby_cols =  []
    feature_list = ['transformers', 'month', 'is_weekend', 'time', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6']
    if target!="y":
        feature_list.remove(target)

    if rank>=1:
        # 一阶
        for i in range(len(feature_list)):
            multi_groupby_cols.append([feature_list[i]])
    if rank>=2:  
        # 二阶
        for i in range(len(feature_list)):
            for j in range(i+1,len(feature_list)):
                multi_groupby_cols.append([feature_list[i],feature_list[j]])
    if rank>=3:
        # 三阶
        for i in range(len(feature_list)-2):
            for j in range(i+1,len(feature_list)-1):
                for h in range(j+1,len(feature_list)):
                    multi_groupby_cols.append([feature_list[i],feature_list[j],feature_list[h]])
    if rank>=4:
        # 四阶
        for x in range(len(feature_list)-3):
            for i in range(x+1,len(feature_list)-2):
                for j in range(i+1,len(feature_list)-1):
                    for h in range(j+1,len(feature_list)):
                        multi_groupby_cols.append([feature_list[x],feature_list[i],feature_list[j],feature_list[h]]) 
    if rank>=5:
        # 五阶
        for x in range(len(feature_list)-4):
            for y in range(x+1,len(feature_list)-3):
                for i in range(y+1,len(feature_list)-2):
                    for j in range(i+1,len(feature_list)-1):
                        for h in range(j+1,len(feature_list)):
                            multi_groupby_cols.append([feature_list[x],feature_list[y],feature_list[i],feature_list[j],feature_list[h]])              
    return multi_groupby_cols
y_multi_groupby_cols = generate_multi_groupby_cols('y',1)
L1_multi_groupby_cols = generate_multi_groupby_cols('L1',2)
L2_multi_groupby_cols = generate_multi_groupby_cols('L2',2)
L3_multi_groupby_cols = generate_multi_groupby_cols('L3',2)
L4_multi_groupby_cols = generate_multi_groupby_cols('L4',2)
L5_multi_groupby_cols = generate_multi_groupby_cols('L5',2)
L6_multi_groupby_cols = generate_multi_groupby_cols('L6',2)
y_multi_groupby_cols = [['transformers'], ['month'], ['is_weekend'], ['time']]

y_multi_groupby_cols

[['transformers'], ['month'], ['is_weekend'], ['time']]

In [12]:
df_train = df[~df[y_cols].isnull()] # 重新分离出训练集和测试集
df_train = df_train.reset_index(drop=True)
df_test = df[df[y_cols].isnull()]

df_train_stas_feat = None
# 为了防止过拟合，采取KFold法统计train训练集中的特征
kfold = KFold(n_splits=5, random_state=random_state, shuffle=True)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train, df_train[y_cols])):
    print(f'\nFold_{fold_id+1} Featuring ==========={time.asctime(time.localtime(time.time()))}\n')
    df_fold_train,df_fold_val = df_train.iloc[trn_idx],df_train.iloc[val_idx] # 划分训练集与验证集
    
    # 计算验证集关于训练集的单/多列组合统计特征
    for cols in tqdm(y_multi_groupby_cols,desc='%-30s'%'y_multi_groupby_cols',leave=False): 
        df_fold_val = multi_groupby(df_fold_train, df_fold_val, cols,y_cols, ['mean'])

    # for cols in tqdm(L1_multi_groupby_cols,desc='%-30s'%'L1_multi_groupby_cols',leave=False): 
    #     df_fold_val = multi_groupby(df_fold_train, df_fold_val, cols,'L1', ['mean'])
    # for cols in tqdm(L2_multi_groupby_cols,desc='%-30s'%'L2_multi_groupby_cols',leave=False): 
    #     df_fold_val = multi_groupby(df_fold_train, df_fold_val, cols,'L2', ['mean'])
    # for cols in tqdm(L3_multi_groupby_cols,desc='%-30s'%'L3_multi_groupby_cols',leave=False): 
    #     df_fold_val = multi_groupby(df_fold_train, df_fold_val, cols,'L3', ['mean'])
    # for cols in tqdm(L4_multi_groupby_cols,desc='%-30s'%'L4_multi_groupby_cols',leave=False): 
    #     df_fold_val = multi_groupby(df_fold_train, df_fold_val, cols,'L4', ['mean'])
    # for cols in tqdm(L5_multi_groupby_cols,desc='%-30s'%'L5_multi_groupby_cols',leave=False): 
    #     df_fold_val = multi_groupby(df_fold_train, df_fold_val, cols,'L5', ['mean'])
    # for cols in tqdm(L6_multi_groupby_cols,desc='%-30s'%'L6_multi_groupby_cols',leave=False): 
    #     df_fold_val = multi_groupby(df_fold_train, df_fold_val, cols,'L6', ['mean'])

    df_train_stas_feat = pd.concat([df_train_stas_feat, df_fold_val], axis=0)# 五折之后拼成一个完整的stas_feat训练集
    # del df_fold_train,df_fold_val
    gc.collect()

# 计算测试集关于训练集的统计特征
for cols in tqdm(y_multi_groupby_cols,desc='%-30s'%'y_multi_groupby_cols',leave=False):
    df_test = multi_groupby(df_train, df_test, cols,y_cols, ['mean'])

# for cols in tqdm(L1_multi_groupby_cols,desc='%-30s'%'L1_multi_groupby_cols',leave=False):
#     df_test = multi_groupby(df_train, df_test, cols,'L1', ['mean'])
# for cols in tqdm(L2_multi_groupby_cols,desc='%-30s'%'L2_multi_groupby_cols',leave=False):
#     df_test = multi_groupby(df_train, df_test, cols,'L2', ['mean'])
# for cols in tqdm(L3_multi_groupby_cols,desc='%-30s'%'L3_multi_groupby_cols',leave=False):
#     df_test = multi_groupby(df_train, df_test, cols,'L3', ['mean'])
# for cols in tqdm(L4_multi_groupby_cols,desc='%-30s'%'L4_multi_groupby_cols',leave=False):
#     df_test = multi_groupby(df_train, df_test, cols,'L4', ['mean'])
# for cols in tqdm(L5_multi_groupby_cols,desc='%-30s'%'L5_multi_groupby_cols',leave=False):
#     df_test = multi_groupby(df_train, df_test, cols,'L5', ['mean'])
# for cols in tqdm(L6_multi_groupby_cols,desc='%-30s'%'L6_multi_groupby_cols',leave=False):
#     df_test = multi_groupby(df_train, df_test, cols,'L6', ['mean'])

df = pd.concat([df_train_stas_feat, df_test], axis=0) # 将分别完成的训练集和测试集的stas_feat合并成最终的df



Fold_1 Featuring ===========Sun Jan 22 18:03:48 2023




Fold_2 Featuring ===========Sun Jan 22 18:03:48 2023




Fold_3 Featuring ===========Sun Jan 22 18:03:48 2023




Fold_4 Featuring ===========Sun Jan 22 18:03:48 2023




Fold_5 Featuring ===========Sun Jan 22 18:03:48 2023



In [13]:
# df
df_test = df[df[y_cols].isnull()].drop(columns=drop_list)
df_train = df[df[y_cols].notnull()].drop(columns=drop_list)


for i,j in zip(df_train.isna().sum().index, df_train.isna().sum()):
    if(j!=0):
        print(i,j)
# 无输出则无空值

In [15]:
# df["L6_y_mean"].info() # .value_counts()
# df.info()
# df_train.isnull().sum()["L6_y_mean"]
df_train[df_train["transformers_y_mean"].isnull()]["y"].values #.value_counts()
# df_train_stas_feat[df_train_stas_feat["L1_y_mean"].isnull()]["y"].values

array([], dtype=float64)

In [14]:
# 使用 `pickle.dump()` 函数将变量保存到文件中
# with open(pickle_dir, 'wb') as f:
#     pickle.dump(df, f)

# with open(pickle_dir, 'rb') as f:
#     df = pickle.load(f)

# 数据训练

In [70]:
# df_train = df[df[y_cols].notnull()].drop(columns=drop_list)
# df_test = df[df[y_cols].isnull()].drop(columns=drop_list)
# df_test.info()

In [13]:
df_train = df.loc[(df['transformers']==0)&(df[y_cols].notnull())].drop(columns=drop_list)
df_test = df.loc[(df['transformers']==0)&(df[y_cols].isnull())].drop(columns=drop_list)
# df_test.info()

In [14]:
oof=[] # 对训练集的预测结果
prediction = df_test[['id']] # 对测试集的预测结果
prediction['y'] = 0
# 排除y和id之后的特征列
feature_names = list(
    filter(lambda x: x not in [y_cols,'id'], df_train.columns))
# feature_names = list_importance_des[:60]
model = lgb.LGBMRegressor(boosting_type='gbdt',metrics='mae',learning_rate=0.01, \
        n_estimators=n_estimators, max_depth=10, num_leaves=295,max_bin=220,min_data_in_leaf=150, \
        feature_fraction= 0.3,bagging_fraction=0.6,bagging_freq=0,lambda_l1=0,lambda_l2=0, \
        min_split_gain=0,verbose=-1,n_jobs=n_jobs,random_state=random_state)

df_importance = pd.DataFrame({'column':feature_names,'importance':0.}) #训练中的特征重要性

kfold = KFold(n_splits=5, random_state=random_state, shuffle=True)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train[feature_names], df_train[y_cols])):
    # 每折一个训练集和验证集
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][y_cols]
    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][y_cols]
    st_time = time.time()
    print(f'\nFold_{fold_id+1} Training ================={time.asctime(time.localtime(time.time()))}\n')
    lgb_model = model.fit(X_train,Y_train,eval_set=[(X_val, Y_val)], verbose=early_stopping_rounds, early_stopping_rounds=early_stopping_rounds)

    pred_val = lgb_model.predict(X_val[feature_names]) # 训练集中每折的验证集
    pred_test = lgb_model.predict(df_test[feature_names]) # 每折对测试集预测，最后取均值

    prediction['y'] += pred_test / 5

    df_oof = df_train.iloc[val_idx][['id', y_cols]].copy()
    df_oof['label'] = pred_val # df_oof是对训练集本身的预测数据
    oof.append(df_oof)

    df_importance['importance'] += (lgb_model.feature_importances_)/5
    del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
    gc.collect()

# del X_test_label,X_test,y
df_oof = pd.concat(oof)
gc.collect()


Fold_1 Training =================Sun Jan 22 05:55:07 2023

[LightGBM] [Warning] feature_fraction is set=0.3, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3
[LightGBM] [Warning] min_data_in_leaf is set=150, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=150
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[1000]	valid_0's l1: 2.45667
[2000]	valid_0's l1: 2.39904
[3000]	valid_0's l1: 2.37714
[4000]	valid_0's l1: 2.36793
[5000]	valid_0's l1: 2.36109


KeyboardInterrupt: 

In [30]:
df_importance.sort_values(by='importance',axis=0,ascending=False).head(10) # 重要性前100的特征
# df_importance.sort_values(by='importance',axis=0,ascending=False).to_csv('/home/mw/project/importance.csv', index=False, columns=['column','importance'])

In [17]:
prediction1 = prediction.copy()

In [18]:
df_train = df.loc[(df['transformers']==1)&(df[y_cols].notnull())].drop(columns=drop_list)
df_test = df.loc[(df['transformers']==1)&(df[y_cols].isnull())].drop(columns=drop_list)
# df_test.info()

In [19]:
oof=[] # 对训练集的预测结果
prediction = df_test[['id']] # 对测试集的预测结果
prediction['y'] = 0
# 排除y和id之后的特征列
feature_names = list(
    filter(lambda x: x not in [y_cols,'id'], df_train.columns))
# feature_names = list_importance_des[:60]
model = lgb.LGBMRegressor(boosting_type='gbdt',metrics='mae',learning_rate=0.01, \
        n_estimators=n_estimators, max_depth=10, num_leaves=295,max_bin=220,min_data_in_leaf=150, \
        feature_fraction= 0.3,bagging_fraction=0.6,bagging_freq=0,lambda_l1=0,lambda_l2=0, \
        min_split_gain=0,verbose=-1,n_jobs=n_jobs,random_state=random_state)

df_importance = pd.DataFrame({'column':feature_names,'importance':0.}) #训练中的特征重要性

kfold = KFold(n_splits=5, random_state=random_state, shuffle=True)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train[feature_names], df_train[y_cols])):
    # 每折一个训练集和验证集
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][y_cols]
    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][y_cols]
    st_time = time.time()
    print(f'\nFold_{fold_id+1} Training ================={time.asctime(time.localtime(time.time()))}\n')
    lgb_model = model.fit(X_train,Y_train,eval_set=[(X_val, Y_val)], verbose=early_stopping_rounds, early_stopping_rounds=early_stopping_rounds)

    pred_val = lgb_model.predict(X_val[feature_names]) # 训练集中每折的验证集
    pred_test = lgb_model.predict(df_test[feature_names]) # 每折对测试集预测，最后取均值

    prediction['y'] += pred_test / 5

    df_oof = df_train.iloc[val_idx][['id', y_cols]].copy()
    df_oof['label'] = pred_val # df_oof是对训练集本身的预测数据
    oof.append(df_oof)

    df_importance['importance'] += (lgb_model.feature_importances_)/5
    del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
    gc.collect()

# del X_test_label,X_test,y
df_oof = pd.concat(oof)
gc.collect()


Fold_1 Training =================Fri Jan 20 15:00:40 2023

[LightGBM] [Warning] feature_fraction is set=0.3, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3
[LightGBM] [Warning] min_data_in_leaf is set=150, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=150
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[1000]	valid_0's l1: 2.32744
[2000]	valid_0's l1: 2.25705
[3000]	valid_0's l1: 2.22737
[4000]	valid_0's l1: 2.21013
[5000]	valid_0's l1: 2.20024
[6000]	valid_0's l1: 2.19272
[7000]	valid_0's l1: 2.19059
[8000]	valid_0's l1: 2.18835
[9000]	valid_0's l1: 2.18567
[10000]

0

In [20]:
prediction2 = prediction.copy()

In [21]:
prediction2

,id,y
43584,M2_D00001_00_00,30.621846
43585,M2_D00001_00_30,31.559642
43586,M2_D00001_01_00,30.471886
43587,M2_D00001_01_30,29.999219
43588,M2_D00001_02_00,29.139461
...,...,...
47995,M2_D00712_21_30,13.376325
47996,M2_D00712_22_00,13.247604
47997,M2_D00712_22_30,14.737235
47998,M2_D00712_23_00,12.526733


In [22]:
prediction = pd.concat([prediction1, prediction2], ignore_index=True)
prediction

,id,y
0,M1_D00000_00_00,9.899992
1,M1_D00000_00_30,9.526917
2,M1_D00000_01_00,8.324575
3,M1_D00000_01_30,8.430197
4,M1_D00000_02_00,8.055017
...,...,...
9595,M2_D00712_21_30,13.376325
9596,M2_D00712_22_00,13.247604
9597,M2_D00712_22_30,14.737235
9598,M2_D00712_23_00,12.526733


In [23]:
prediction.to_csv(prediction_dir, index=False, columns=['id','y'])

## 调参

In [25]:
df_train = df[df[y_cols].notnull()].drop(columns=drop_list)
df_test = df[df[y_cols].isnull()].drop(columns=drop_list)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48000 entries, 1 to 47999
Columns: 696 entries, id to L4_L5_L6_mean
dtypes: float64(672), int32(3), int64(20), object(1)
memory usage: 254.7+ MB


In [38]:
feature_names = list(
    filter(lambda x: x not in [y_cols,'id'], df_train.columns))
    
def  my_gridsearch():
    highest = 0. #
    for i in range(12,19,1):
        #每个i对于8个j
        score_pre = 0.
        print("————————————————————⬇\t当前 i：%d\t⬇——————————————————————"%(i))
        for j in range(20,400,20):
            start=datetime.datetime.now()
            model = lgb.LGBMRegressor(boosting_type='gbdt',metrics='mae',learning_rate=0.01, \
        n_estimators=5000, max_depth=i, num_leaves=j,max_bin=220,min_data_in_leaf=150, \
        feature_fraction= 0.3,bagging_fraction=0.6,bagging_freq=0,lambda_l1=0,lambda_l2=0, \
        min_split_gain=0,verbose=-1,n_jobs=n_jobs,random_state=random_state)

            score = abs(cross_val_score(model,df_train[feature_names], df_train[y_cols],cv=5,scoring="neg_mean_absolute_error").mean())
            del model
            gc.collect()
            print(score)
            if score > highest :
                highest =  score
                print("New Highest \tScore!=%s\t\tWhen\ti=%d\t\tj=%d"%(score,i,j))
            elif score <= highest:
                print("i=%d\tj=%d\tscore=%s"%(i,j,score))
                if score == score_pre: #如果本次score与上次一样则跳出本次循环（在粗调节时使用）
                    break
            end=datetime.datetime.now()
            score_pre = score
            print('⬆Running time: %s Seconds'%(end-start))
if __name__ == '__main__':
    my_gridsearch();
    print("End!")
# i=11	j=200	score=2.3175

————————————————————⬇	当前 i：12	⬇——————————————————————
[LightGBM] [Warning] feature_fraction is set=0.3, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3
[LightGBM] [Warning] min_data_in_leaf is set=150, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=150
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=0.3, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3
[LightGBM] [Warning] min_data_in_leaf is set=150, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=150
[LightGBM] [Warning] baggi